In [16]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.max_rows = 120
import matplotlib as mpl
import sys
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import scipy

<Figure size 640x480 with 0 Axes>

In [17]:
dataset_material = pd.read_excel('Data_Ghiringhelli.xlsx', 'Material Data')
dataset_atomic = pd.read_excel('Data_Ghiringhelli.xlsx',   'Atomic Data')
dataset_atomic.columns = ['Z', 'A', 'IP', 'EA', 'HOMO', 'LUMO', 'rs', 'rp', 'rd', '1D']

In [18]:
def get_row(element,dataframe):
    return (dataframe.loc[dataframe['A']==element])

def collecting_descriptor(a,b, dataframe=dataset_atomic):
    row_a = get_row(element=a, dataframe=dataframe)
    row_b = get_row(element=b, dataframe=dataframe)
    return row_a,row_b

In [19]:
def formula(a,b):
    return [(a['rs'].values**2 + np.exp(a['rs'].values))/a['rs'].values**2 , (a['rd'].values - b['rp'].values**3) / a['rp'].values**3]

In [20]:
#('(rp_B**3 - exp(rp_A))/((rp_A**3))', '(rp_A**2 -sqrt(fabs(rd_A)))/(exp(rp_A))')
def formula(a,b):
    return [(b['rp'].values**3 - np.exp(a['rp'].values))/a['rp'].values**3 / (a['rp'].values**2-np.sqrt(abs(a['rd'].values)))/np.exp(a['rp'].values)]

In [25]:
def formula(a,b):
    return [(b['IP'].values-b['EA'].values)/a['rp'].values**2, abs(a['rs'].values-b['rp'].values)/np.exp((a['rs'].values))]

In [26]:
def cross_check(dataframe=dataset_material, y_array=dataset_material['DE'].values):
    feature_array, val1, val2 = [],[],[]
    for kk in range(len(dataframe.values)):
        el_a,el_b   = (dataframe.values)[kk][2:4]
        row_a,row_b = collecting_descriptor(a=el_a, b=el_b)
        res = formula(row_a, row_b)
        feature_array.append(res)
    return feature_array

In [27]:
aar = np.array(cross_check())
n_samp, nx, ny = np.shape(aar)
feature_array = aar.reshape((n_samp,nx*ny))

In [28]:
mse = []
for ii in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(feature_array,dataset_material['DE'].values, test_size=0.1, random_state=ii)
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    mse.append(mean_squared_error(y_test,y_pred))
avg = float(np.average(mse))
print('avg rmse = ',avg)

avg rmse =  0.010840896048690563


In [35]:
for i in range(len(feature_array[0])):
    val_1, val_2 = scipy.stats.pearsonr(feature_array[:,i], dataset_material['DE'].values)
    print(i+1,'pearson_coeff :- ',val_1)
    print(i+1,'p_value :- ',val_2)

1 pearson_coeff :-  0.8982560327555467
1 p_value :-  2.6848323416519622e-30
